# Proyecto Final Humanidades Digitales - FSalasNoguera

Este script hace peticiones al API de Spotify

1. ### Se importan las librerias y se pone el Token de Auth para el API de Spotify



In [19]:
#Importación de librerías
import datetime
import requests
import csv
import json
import random
import pandas as pd
import csv
from google.colab import files



# Timestamp como identificador de los archivos CSV
d = datetime.datetime.now()
timestamp = d.strftime("%Y-%m-%dT%H%M%S")
print(timestamp)


# Token de auth
token = 'BQDfN_rFGKp1d4w4splx568pBhqGYcB-wkFZTiELaTaL5hBM_VXrQwUS0vhb5VLql90SfZdDq46mj0F_hNtFeFnhIPvkgfzM62cUntrBN50s-3RCJHnP3r3oiO-eDtZComixXseqzhV31s-wk9educG7Zis5NRxj8Ro3TGzmqtw71HJMFq675aj2kzsvv40'
username = "felipesalas1234"

# Para obtener el Auth toca usar oAuth2.0, esto lo estoy haciendo desde Postman



2021-06-04T221300


2. ### Ingresa por teclado el nombre de un artista y lo busca en Spotify 

In [20]:
#URL base del API

baseUrl = "https://api.spotify.com/v1/"

#Pide un input para buscar a un artista

artistNameInput = input()

id = 0


##https://api.spotify.com/v1/search?q=artistNameInput&type=artist&limit=1

url = "https://api.spotify.com/v1/" + "search?q=" + artistNameInput + "&type=artist&limit=1"



payload={}
headers = {
  'Accept': 'application/json',
  'Authorization': 'Bearer ' + token
}

response = requests.request("GET", url, headers=headers, data=payload)

#Si la respuesta del servidor es 200 o que fue satisfactoria almacena la respuesta JSON en una variable

if response.status_code == 200:

    searchArtist = response.json()

else: print(response,status_code)

#Obtiene el nombre del artista, su popularidad y ID
artists = searchArtist['artists']
theArtist = artists['items']
searchArtistName = theArtist[0]
theArtistName = searchArtistName['name']
theArtistId = searchArtistName['id']
theArtistPop = searchArtistName['popularity'] 

print("El artista ingresado es " + theArtistName)
print("Su popularidad es " + str(theArtistPop))
print("Su popularidad es " + str(theArtistId))

id = theArtistId

#Añade el nombre del artista en una variable para usar más adelante
artista = theArtistName


print("El artista ingresado es " + artista)

#Nombre del archivo de salida
csv_out = artista + "TracksAndRelated" + timestamp + ".csv"



#print(response.text)
    

the national
El artista ingresado es The National
Su popularidad es 71
Su popularidad es 2cCUtGK9sDU2EoElnk0GNB
El artista ingresado es The National


### Opcional, buscar el artista por ID

In [ ]:

#Opcional, Obtener info del artista por ID

id = input()

url = "https://api.spotify.com/v1/" + "artists/" + id

payload={}
headers = {
  'Accept': 'application/json',
  'Authorization': 'Bearer ' + token
}

response = requests.request("GET", url, headers=headers, data=payload)

if response.status_code == 200:

    artist = response.json()

else: print(response,status_code)

  # Store the artist name in the environment
artista = artist['name']


print("El artista ingresado es " + artista)



3. ### Obtenemos artistas relacionados del artista

In [21]:
#URL para hacer el request GET para obtener los artistas relacionados con el artista que se ingreso anteriormente
url = "https://api.spotify.com/v1/artists/" + id + "/related-artists"

payload={}
headers = {
  'Accept': 'application/json',
  'Authorization': 'Bearer ' + token
}

response = requests.request("GET", url, headers=headers, data=payload)


if response.status_code == 200:

    responseRelated = response.json()
    
else: print(response,status_code)

#contador para el for y Lista de los artistas relacionados
count = 0
idRelated = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

for artistNum in responseRelated['artists']:
    artistsNames = artistNum['name']
    artistsIds = artistNum['id']
    #Imprime cada artista relacionado que informa Spotify
    print(artistsNames)
    idRelated [count] = artistNum['id']
    count += 1
    
    


            



EL VY
Arcade Fire
Band of Horses
Okkervil River
Frightened Rabbit
Wolf Parade
Clap Your Hands Say Yeah
The Walkmen
The Antlers
Broken Social Scene
Matt Berninger
TV On The Radio
Phosphorescent
Big Thief
The War On Drugs
Father John Misty
Grizzly Bear
Editors
Fleet Foxes
Big Red Machine


4. ### Se crea un playlist en la cuenta del usuario

In [26]:
#URL para el request POST para crear la playlist en el usuario

url = "https://api.spotify.com/v1/users/"+ username +"/playlists"

payload = json.dumps({
  "name": artista + " Mix",
  "public": False
})
headers = {
  'Accept': 'application/json',
  'Authorization': 'Bearer '+token,
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

responsePlaylist = response.json()

#Obtiene los datos del JSON y los almacena en las variables
elNombre = responsePlaylist['name']
playlistId = responsePlaylist['id']

print("El nombre de la playlist creada es: " + elNombre)



El nombre de la playlist creada es: The National Mix


5. ### Toma los Top Tracks del artista y los artistas relacionados. crea un CSV con los datos y los prepara para enviarlos a spotify





In [23]:
#Contador para el for de las canciones
count = 0

#Pepara el CSV para guardar los datos obtenidos de Spotify
with open(csv_out, 'w') as csvfile:

  
  #Instanciación del objeto writer de la librería CSV
  tablewriter = csv.writer(csvfile, delimiter=',', quotechar='"')
  

  # Headers para el CSV
  headers = ["track_title","track_id","track_artist","track_pop","track_uri","track_rDate"]

   # Se escriben los encabezados en la tabla
  tablewriter.writerow(headers)

  # campos a buscar en los metadatos
  track_title = ""
  track_id = ""
  track_artist = ""
  track_pop = ""
  track_uri = ""
  track_rDate = ""



  #Toma los top tacks de las bandas relacionadas
  #Contadores para el For y los IF
  countb = 0
  counta = 0
  #Lista de los tracks relacionados, crea 90 y los llena con un dato Dummie
  idTracks = ["spotify:track:3ZLElfsFXSMmNCliIGhIb7"] * 90

  #Para cada artista relacionado hace una petición para obtener sus top tracks
  for band in idRelated:
      url = "https://api.spotify.com/v1/artists/" + idRelated[countb] + "/top-tracks?country=co"
      payload={}
      headers = {
      'Accept': 'application/json',
      'Authorization': 'Bearer '+token,
      'Content-Type': 'application/json'
      }

      response = requests.request("GET", url, headers=headers, data=payload)

    
      
      if response.status_code == 200:

        responseTracks = response.json()
      
      else: print(response.status_code)

      
     #contador para el for
      countd = 0
      for trackId in responseTracks['tracks']:
        

        #Toma datos del Json
        track_title = trackId['name']
        print(track_title)
        track_id = trackId['id']
        countc = 0
        for trackArt in trackId['artists']:
          
          if countc > 0:
            track_artist = track_artist + " - " + trackArt['name']
          else:
            track_artist = trackArt['name']
          countc += 1
          print(track_artist)
        track_pop = trackId['popularity']
        track_uri = trackId['uri']
        track_rDate = trackId['album']['release_date']

        new_row = [track_title,track_id,track_artist,track_pop,track_uri,track_rDate]

        #escribimos la fila en la tabla
        tablewriter.writerow(new_row) 

        print(countd)

        #toma solamente tres tracks para añadirlo a la playlist creada los otros datos son solamente para el CSV
        if countd <= 3:
        
          #trackIds = trackId['id']
          idTracks [counta] = trackId['uri']
          countd += 1
          counta += 1
        else: print("Artista número: " + str(countb))
        

      countb += 1 

  print(idTracks)

  #Añade las canciones del artista principal

  url = "https://api.spotify.com/v1/artists/" + id + "/top-tracks?country=co"
  payload={}
  headers = {
  'Accept': 'application/json',
  'Authorization': 'Bearer '+token,
  'Content-Type': 'application/json'
  }

  response = requests.request("GET", url, headers=headers, data=payload)

    
      
  if response.status_code == 200:

      responseTracks = response.json()

  else: print(response.status_code)

      
  #Añade las canciones del artista principal a la lista de tracks  
  for trackId in responseTracks['tracks']:
      #trackIds = trackId['id']
      idTracks [counta] = trackId['uri']
      track_title = trackId['name']
      print(track_title)
      track_id = trackId['id']
      countc = 0
      for trackArt in trackId['artists']:
          
        if countc > 0:
          track_artist = track_artist + " - " + trackArt['name']
        else:
          track_artist = trackArt['name']
        countc += 1
        print(track_artist)
        track_pop = trackId['popularity']
        track_uri = trackId['uri']
        track_rDate = trackId['album']['release_date']

        new_row = [track_title,track_id,track_artist,track_pop,track_uri,track_rDate]

        #escribimos la fila en la tabla
        tablewriter.writerow(new_row) 
      counta += 1

files.download('/content/'+ artista + 'TracksAndRelated' + timestamp + '.csv')


Return to the Moon (Political Song for Didi Bloome to Sing, with Crescendo)
EL VY
0
Happiness, Missouri
EL VY
1
Need a Friend
EL VY
2
Paul Is Alive
EL VY
3
No Time to Crank the Sun
EL VY
4
Artista número: 0
I'm the Man to Be
EL VY
4
Artista número: 0
It's a Game
EL VY
4
Artista número: 0
Careless
EL VY
4
Artista número: 0
Silent Ivy Hotel
EL VY
4
Artista número: 0
Are These My Jets (30 Days, 30 Songs)
EL VY
4
Artista número: 0
Wake Up
Arcade Fire
0
Rebellion (Lies)
Arcade Fire
1
Everything Now
Arcade Fire
2
The Suburbs
Arcade Fire
3
My Body Is a Cage
Arcade Fire
4
Artista número: 1
Put Your Money on Me
Arcade Fire
4
Artista número: 1
Reflektor
Arcade Fire
4
Artista número: 1
Ready To Start
Arcade Fire
4
Artista número: 1
Afterlife
Arcade Fire
4
Artista número: 1
Neighborhood #1 (Tunnels)
Arcade Fire
4
Artista número: 1
The Funeral
Band of Horses
0
No One's Gonna Love You
Band of Horses
1
No One's Gonna Love You (Stockholm Version)
Band of Horses
2
Is There A Ghost
Band of Horses
3
The 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

6. ### Desorganiza los items y los añade al playlist

In [25]:
#Random el orden las pistas de la playlist

random.shuffle(idTracks)

print(idTracks)

#Prepara los URIS para que Spotify lo acepte

uris = ",".join(idTracks)

print(uris)

url = "https://api.spotify.com/v1/users/felipesalas1234/playlists/" + playlistId + "/tracks?uris=" + uris

payload={}
headers = {'Accept': 'application/json','Authorization': 'Bearer '+token,'Content-Type': 'application/json'}

response = requests.request("POST", url, headers=headers, data=payload)


    
if response.status_code == 200:

    responseTracks = response.json()
else: print(response.status_code)

print("se ha creado la playlist en la cuenta de: " + username)

['spotify:track:1RvaEm9T7sDm5yjDezbDGa', 'spotify:track:7hYiX6LMP8w8d0kEc4KWuW', 'spotify:track:202QyrB6Q3Kimsr7KqJut3', 'spotify:track:3wfF0Nw234zHY1vdIWIc2g', 'spotify:track:2x24uEfWPzORC6r7FdjBFz', 'spotify:track:4Sd5tbNP4q8wS64abqwfkN', 'spotify:track:2ioYU4ERcuqscet68UCMld', 'spotify:track:7zxikeAavSLQVGocbK2nau', 'spotify:track:1K5wIDgNBgT7lVOMibiRAE', 'spotify:track:70ZuQywnmOpqcIiEnUA5yV', 'spotify:track:3ZHdHRSRPgqAheKwck2KgI', 'spotify:track:5cf1ZfI16kHp3S1ySpQ36G', 'spotify:track:3nmIdg9K8BciyCky9D4tVD', 'spotify:track:3Bm9CsRaCg4FDRfnX0XW71', 'spotify:track:45D0ixQWzXQOoU1S9aSnKk', 'spotify:track:3cabEB5HG1oocs574S4icH', 'spotify:track:371akJ9fYgSkrAFMyWoTdL', 'spotify:track:7LB1G3aIem48mHFQrDjdaV', 'spotify:track:3nKNCpvNgue4dRPqPbywU8', 'spotify:track:7KsZHCfOitA5V9oQYVdltG', 'spotify:track:3YOv5LNRVbrmWnVVyk3Flj', 'spotify:track:0Z57YWES04xGh3AImDz6Qr', 'spotify:track:2IvNxLl01CTAfCOA103Tgx', 'spotify:track:6h4V0fuWElsTFaPg2tQIoL', 'spotify:track:6KXbSJLz6h8NvysBPm8ctt',